# Tugas Pemrograman Kecil 3

# Chapter 1: Embedding with SVD & LSA

Pengantar Singular Value Decomposition (SVD) dan Latent Semantic Analysis (LSA)

SVD adalah metode aljabar untuk memecah matriks menjadi komponen-komponen yang lebih sederhana. Metode ini memungkinkan kita untuk menyederhanakan data yang kompleks tanpa kehilangan informasi penting.

LSA, di sisi lain, menggunakan SVD untuk menganalisis hubungan antara kata-kata dan dokumen dalam korpus. Hasil dari LSA dapat kita jadikan sebagai embedding, baik itu `document embeddings` atau `word embeddings`

Dalam tutorial ini, kita akan mempelajari dasar-dasar matematika di balik SVD dan LSA, serta bagaimana menerapkannya menggunakan Python. Kita juga akan melihat bagaimana teknik-teknik ini dapat digunakan untuk mengurangi dimensi data dan menemukan hubungan semantik dalam teks.

In [58]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

np.random.seed(42)

## Singular Value Decomposition (SVD)

   SVD adalah sebuah teknik dalam aljabar linier yang menguraikan sebuah matriks menjadi tiga matriks komponen. Untuk setiap matriks $A \in \mathbb{R}^{m \times n}$, SVD menyatakannya sebagai:

   $A = U \Sigma V ^ T $

   Di mana:
   - $ U \in \mathbb{R}^{m \times m}$ adalah matriks ortogonal yang berisi vektor singular kiri
   - $\Sigma \in \mathbb{R}^{m \times n}$ adalah matriks diagonal dengan nilai tunggal
   - $V^T \in \mathbb{R}^{n \times n}$ adalah transpose dari matriks ortogonal yang berisi vektor singular kanan

### Signifikansi Komponen SVD:
   - Entri diagonal $\Sigma$ (nilai singular) mewakili pentingnya setiap dimensi
   - Nilai singular berhubungan dengan nilai eigen: $\sigma_i = \sqrt{\lambda_i}$
   - $\lambda_i$ adalah nilai eigen dari $A^TA$ atau $AA^T$

### Proses Komputasi SVD:
   1. Bentuklah matriks covariance C
   2. Hitung nilai eigen dan vektor eigen dari C
   3. Dapatkan nilai-nilai singular: $\sigma_i = \sqrt{\lambda_i}$
   4. Buat V dari vektor-vektor eigen dari C
   5. Hitung U: $U_i = \frac{1}{\sigma_i}AV_i$

### SVD Truncated dan Low-Rank Approximation:
   SVD memungkinkan reduksi dimensi dengan melakukan aproksimasi A dengan dimensi yang lebih sedikit:
   
   $A_k = \sum_{i=1}^k \sigma_i u_i v_i^T$
   
   k-rank approximation ini masih bisa menangkap fitur-fitur terpenting dari matriks asli.

### Task 1
Implementasikan SVD from scratch dengan hanya memanfaatkan numpy <font color='red'>**tanpa function SVD dari luar** </font>.

Feel free untuk mengikuti algoritma dibawah ini, kalau ada cara sendiri juga silakan.

1. Deklarasi fungsi SVD:
   - Input: Matriks A dan parameter opsional k (jumlah singular value yang akan dihitung)
   - Ubah A menjadi array NumPy dan tentukan shapenya (n x m)
   - Ubah k ke min(n, m) jika tidak ditentukan, atau min(k, n, m) jika ada

2. Hitung matriks covariance:
   - Jika n >= m, hitung C = A^T * A
   - Jika n < m, hitung C = A * A^T

3. Menghitung nilai eigen dan vektor eigen:
   - Gunakan `np.linalg.eigh(C)` untuk menghitung nilai eigen dan vektor eigen dari C
   - Urutkan nilai eigen dan vektor eigen secara descending

4. Hitung nilai singular:
   - Ambil akar kuadrat dari k nilai eigen pertama

5. Hitung vektor-vektor singular kanan:
   - Tetapkan V sebagai k kolom pertama dari vektor eigen yang telah diurutkan
   - Jika n < m, update V dengan rumus: `V = A^T * V / singular_values` dan set U menjadi V.
     - Return U, matriks diagonal dari singular values, dan V^T
     - Lihat dibawah kenapa bisa begini kalau penasaran

6. Hitung vektor singular kiri:
   - Hitung U dengan `U = A^T * V / singular_values`

7. Return U, matriks diagonal dari singular values, dan V^T

In [59]:
def svd(A, k=None):
    A = np.array(A, dtype=float)
    n, m = A.shape
    k = min(n, m) if k is None else min(k, n, m)

    # TODO

    # 2. Menghitung matrix covariance
    if n >= m:
      C = A.T @ A
    else:
      C = A @ A.T

    # 3 .Menghitung vector eigen dan nilai eigen (labmda)
    eigen_values, eigen_vectors = np.linalg.eigh(C)

    idx = np.argsort(eigen_values)[::-1]
    eigen_values = eigen_values[idx]
    eigen_vectors = eigen_vectors[:, idx]

    # 4. Hitung nilai singular
    singular_values = np.sqrt(eigen_values[:k])

    # 5. Hitung Vektor singular kanan
    V = eigen_vectors[:, :k]

    if n >= m:
        U = (A @ V) / singular_values
    else:
        U = V
        V = (A.T @ U) / singular_values

    return U, np.diag(singular_values), V.T


### Informasi tambahan untuk step 5

Dalam SVD, kita mendekomposisi matriks A (n x m) menjadi U (n x k), S (k x k), dan V^T (k x m), di mana k adalah jumlah nilai singular yang kita hitung.

Ketika n < m, kita awalnya menghitung vektor eigen dari AA^T alih-alih A^TA. Mari kita sebut vektor eigen ini U_init. Vektor eigen ini berkorespondensi dengan vektor singular kiri A, tapi bukan vektor singular kanan yang kita butuhkan untuk V.

Berikut alasan mengapa rumus `V = A^T * V / singular_values` bekerja:

1. Dalam SVD, kita memiliki hubungan: $AV = US$

2. Bisa diapakan A^T: $A^TAV = A^TUS$

3. Ingat bahwa V seharusnya adalah vektor eigen dari A^TA. Jadi, $A^TAV = V\Sigma^2$, di mana $\Sigma^2$ adalah matriks diagonal dari nilai eigen.

4. Dari langkah 2 dan 3: $A^TUS = V\Sigma^2$

5. Bagi kedua sisi dengan $\Sigma$ (matriks diagonal dari nilai singular): $(A^TU)\Sigma^{-1} = V$

6. Ini setara dengan rumus kita: $V = \frac{A^T \cdot U}{\text{singular values}}$

Dalam implementasi kita, awalnya kita menghitung U_init (vektor eigen dari AA^T) alih-alih U. Tapi U_init sebenarnya sama dengan U, jadi kita bisa menggunakannya sebagai pengganti U dalam rumus.

Jadi, ketika kita menghitung $V = \frac{A^T \cdot U}{\text{singular values}}$, kita secara efektif mengubah vektor singular kiri (yang kita hitung sebagai vektor eigen dari AA^T) menjadi vektor singular kanan.

Pendekatan ini memungkinkan kita untuk menghitung V yang benar bahkan ketika kita mulai dengan AA^T alih-alih A^TA, memastikan bahwa SVD kita bekerja dengan benar untuk kedua kasus n >= m dan n < m.

## Latent Semantic Analysis (LSA)

   LSA menerapkan SVD untuk menganalisis hubungan antara dokumen dan term dalam korpus:
   1. Membangun matriks istilah-dokumen A menggunakan pembobotan TF-IDF
   2. Lakukan SVD pada
   
   $A = U\Sigma V^T$

   3. Reduksi dimensi dengan mengambil hanya k nilai/vektor singular teratas

### TF-IDF
   TF-IDF mengukur tingkat kepentingan term dalam sebuah dokumen relatif terhadap sebuah korpus:

   $\text{TF-IDF}(t,d,D) = \text{TF}(t,d) \cdot \text{IDF}(t,D)$
   
   Dimana:

   - $\text{TF}(t,d) = \frac{\text{jumlah term t di dokumen d}}{\text{total term di d}}$
   - $\text{IDF}(t,D) = \log\frac{\text{total dokumen}}{\text{dokumen yang mengandung t}}$

### Menafsirkan Hasil LSA:
   Setelah menerapkan SVD:
   - Baris $U\Sigma_k$ mewakili dokumen dalam ruang semantik yang telah direduksi
   - Kolom $V^T$ mewakili terms dalam ruang ini
   Kemiripan antara dokumen atau istilah dapat diukur dengan menggunakan kesamaan kosinus dalam ruang yang direduksi ini.

### Task 2
Implementasikan LSA dengan implementasi SVD kalian sebelumnya.

Feel free untuk mengikuti algoritma dibawah ini, kalau ada cara sendiri juga silakan.

1. Deklarasi fungsi LSA:
   - Input: `documents` dan `num_topics`

2. Lakukan vektorisasi TF-IDF:
    - Silakan gunakan `TfidfVectorizer` dari `sklearn`

3. Terapkan SVD ke matriks TF-IDF:
    - Panggil fungsi SVS sebelumnya dengan matriks tfidf sebelumnya dan `k = num_topics`

4. Hitung matriks:
    - Matriks document-topic: U * S
    - Matriks term-topic: V

In [60]:
def lsa(documents, num_topics):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents).toarray()
    terms = vectorizer.get_feature_names_out()

    # TODO
    U, S, VT = svd(tfidf_matrix, num_topics)

    doc_topic_matrix = U @ S
    term_topic_matrix = VT

    # U = U[:, :num_topics]
    # S = np.diag(S[:num_topics])
    # VT = VT[:num_topics, :]

    # doc_topic_matrix = VT.T @ S

    # term_topic_matrix = U @ S

    return doc_topic_matrix, term_topic_matrix, terms


Untuk mendapatkan embeddingnya, kita dapat menggunakan matriks yang dihasilkan oleh LSA. Jangan lupa untuk men-transpose term matriksnya.

In [61]:
documents = [
    # SpongeBob SquarePants
    "SpongeBob lives in a pineapple under the sea in Bikini Bottom.",
    "Patrick Star is SpongeBob's best friend and lives under a rock.",
    "Squidward Tentacles is SpongeBob's grumpy neighbor who plays the clarinet.",

    # Computer Science
    "Python is a popular programming language known for its simplicity and readability.",
    "Algorithms are step-by-step procedures for solving computational problems.",
    "Data structures like arrays and linked lists are fundamental in computer science.",

    # Cooking 🔥🔥🔥
    "Sautéing involves cooking food quickly in a small amount of oil over high heat.",
    "Baking is a cooking method that uses dry heat, typically in an oven.",
    "Seasoning with herbs and spices can greatly enhance the flavor of dishes."
]

doc_topic_matrix, term_topic_matrix, terms = lsa(documents, 3)

print(doc_topic_matrix)
print(term_topic_matrix)

document_embeddings = doc_topic_matrix
print("Document embeddings:")
for i, embedding in enumerate(document_embeddings):
    print(f"Document {i}: {embedding}")

word_embeddings = term_topic_matrix
word_embedding_dict = dict(zip(terms, word_embeddings))
print("\nWord embeddings:")
for word, embedding in word_embedding_dict.items():
    print(f"{word}: {embedding}")

[[-0.65503505  0.08756422 -0.23535281]
 [-0.59749834  0.37524266 -0.13820511]
 [-0.42550279  0.30898413 -0.22702776]
 [-0.26376106  0.21932515  0.50295872]
 [-0.07958578  0.07452861  0.70868065]
 [-0.3018436  -0.08110509  0.45279548]
 [-0.3235538  -0.70737842 -0.01379984]
 [-0.39291898 -0.5706551  -0.01102955]
 [-0.28178756  0.00269099  0.05822144]]
[[-0.01598763 -0.06461839 -0.08326329 -0.21075079 -0.06678949 -0.06308913
  -0.08326329 -0.14520269 -0.15543487 -0.15543487 -0.01598763 -0.05856991
  -0.09910895 -0.01598763 -0.06308913 -0.12490322 -0.06308913 -0.05856991
  -0.08326329 -0.05856991 -0.05856991 -0.06461839 -0.06244244 -0.14520269
  -0.06308913 -0.05856991 -0.09910895 -0.12490322 -0.05856991 -0.06461839
  -0.33859892 -0.06461839 -0.25014487 -0.05794236 -0.05794236 -0.05794236
  -0.06308913 -0.06308913 -0.06308913 -0.25392327 -0.08326329 -0.09910895
  -0.1040468  -0.06461839 -0.08326329 -0.06461839 -0.14520269 -0.15543487
  -0.09910895 -0.05794236 -0.01598763 -0.01598763 -0.057

## Test

Jalankan test dibawah ini untuk mengetahui apakah implementasi kalian sudah benar atau belum.

In [62]:
#@title Algoritma Test

from scipy.linalg import svd as scipy_svd
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
class Colors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'

def print_color(text, color):
    print(f"{color}{text}{Colors.ENDC}")

def test_svd():
    print_color("Testing SVD implementation...", Colors.HEADER)

    A = np.random.rand(5, 4)
    U, S, V = svd(A)
    U_scipy, S_scipy, V_scipy = scipy_svd(A, full_matrices=False)

    print_color("\nTest 1: Comparison with scipy's SVD", Colors.BOLD)

    max_diff_U = np.max(np.abs(np.abs(U) - np.abs(U_scipy)))
    max_diff_S = np.max(np.abs(np.diag(S) - S_scipy))
    max_diff_V = np.max(np.abs(np.abs(V) - np.abs(V_scipy)))

    print(f"Max difference in U: {max_diff_U:.6f}")
    print(f"Max difference in S: {max_diff_S:.6f}")
    print(f"Max difference in V: {max_diff_V:.6f}")

    tolerance = 1e-4
    if max(max_diff_U, max_diff_S, max_diff_V) < tolerance:
        print_color("Test 1 Passed!", Colors.OKGREEN)
    else:
        print_color("Test 1 Failed: Large differences detected", Colors.FAIL)

    A_reconstructed = np.dot(U, np.dot(S, V))
    reconstruction_error = np.linalg.norm(A - A_reconstructed)
    print_color("\nTest 2: Reconstruction Error", Colors.BOLD)
    print(f"Reconstruction error: {reconstruction_error:.6f}")

    if reconstruction_error < 1e-5:
        print_color("Test 2 Passed!", Colors.OKGREEN)
    else:
        print_color("Test 2 Failed: Large reconstruction error", Colors.FAIL)

    print_color("\nTest 3: Small, known matrix", Colors.BOLD)

    A_small = np.array([[1, 2], [3, 4], [5, 6]])
    U_small, S_small, V_small = svd(A_small)

    U_scipy, S_scipy, V_scipy = scipy_svd(A_small, full_matrices=False)

    max_diff_U = np.max(np.abs(np.abs(U_small) - np.abs(U_scipy)))
    max_diff_S = np.max(np.abs(np.diag(S_small) - S_scipy))
    max_diff_V = np.max(np.abs(np.abs(V_small) - np.abs(V_scipy.T)))

    tolerance = 1e-6

    print(f"\nMax difference in U: {max_diff_U:.6f}")
    print(f"Max difference in S: {max_diff_S:.6f}")
    print(f"Max difference in V: {max_diff_V:.6f}")

    if max(max_diff_U, max_diff_S, max_diff_V) < tolerance:
        print_color("Test 3 Passed: Small matrix SVD matches scipy results within tolerance", Colors.OKGREEN)
    else:
        print_color("Test 3 Failed: Large differences detected in small matrix SVD", Colors.FAIL)

    U_orthogonality = np.dot(U.T, U)
    V_orthogonality = np.dot(V, V.T)
    print_color("\nTest 4: Orthogonality of U and V", Colors.BOLD)

    U_ortho_error = np.max(np.abs(U_orthogonality - np.eye(U_orthogonality.shape[0])))
    V_ortho_error = np.max(np.abs(V_orthogonality - np.eye(V_orthogonality.shape[0])))

    print(f"Max deviation from identity for U: {U_ortho_error:.6f}")
    print(f"Max deviation from identity for V: {V_ortho_error:.6f}")

    if max(U_ortho_error, V_ortho_error) < 1e-6:
        print_color("Test 4 Passed!", Colors.OKGREEN)
    else:
        print_color("Test 4 Failed: U or V not orthogonal", Colors.FAIL)

    print_color("\nTest 5: Properties of singular values", Colors.BOLD)
    singular_values = np.diag(S)
    print("Singular values:")
    print(singular_values)

    if np.all(singular_values >= 0) and np.all(np.diff(singular_values) <= 0):
        print_color("Test 5 Passed!", Colors.OKGREEN)
    else:
        print_color("Test 5 Failed: Singular values not non-negative or not in descending order", Colors.FAIL)

def test_lsa():
    print_color("\nTesting LSA implementation...", Colors.HEADER)

    documents = [
        # SpongeBob SquarePants
        "SpongeBob lives in a pineapple under the sea in Bikini Bottom.",
        "Patrick Star is SpongeBob's best friend and lives under a rock.",
        "Squidward Tentacles is SpongeBob's grumpy neighbor who plays the clarinet.",

        # Computer Science
        "Python is a popular programming language known for its simplicity and readability.",
        "Algorithms are step-by-step procedures for solving computational problems.",
        "Data structures like arrays and linked lists are fundamental in computer science.",

        # Cooking
        "Sautéing involves cooking food quickly in a small amount of oil over high heat.",
        "Baking is a cooking method that uses dry heat, typically in an oven.",
        "Seasoning with herbs and spices can greatly enhance the flavor of dishes."
    ]

    num_topics = 3

    doc_topic_matrix, term_topic_matrix, terms = lsa(documents, num_topics)

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents)
    lsa_sklearn = TruncatedSVD(n_components=num_topics, random_state=42)
    doc_topic_matrix_sklearn = lsa_sklearn.fit_transform(tfidf_matrix)

    print_color("\nOur LSA document-topic matrix:", Colors.BOLD)
    print(doc_topic_matrix)
    print_color("\nsklearn's LSA document-topic matrix:", Colors.BOLD)
    print(doc_topic_matrix_sklearn)

    shape_match = doc_topic_matrix.shape == doc_topic_matrix_sklearn.shape
    print_color(f"\nShape match: {shape_match}", Colors.OKGREEN if shape_match else Colors.FAIL)

    our_distances = np.linalg.norm(doc_topic_matrix[0] - doc_topic_matrix[1:], axis=1)
    sklearn_distances = np.linalg.norm(doc_topic_matrix_sklearn[0] - doc_topic_matrix_sklearn[1:], axis=1)
    relative_diff = np.abs(our_distances / our_distances.sum() - sklearn_distances / sklearn_distances.sum())

    print_color(f"\nMax relative difference in document distances: {np.max(relative_diff):.6f}",
                Colors.OKGREEN if np.max(relative_diff) < 0.1 else Colors.FAIL)

    print_color("\nTop terms for each topic (our implementation):", Colors.BOLD)
    for topic in range(num_topics):
        top_terms = sorted([(terms[i], term_topic_matrix[topic, i]) for i in range(len(terms))],
                           key=lambda x: abs(x[1]), reverse=True)[:5]
        print(f"\nTopic {topic + 1}:")
        for term, weight in top_terms:
            print(f"{term}: {weight:.4f}")

    print_color("\nLSA Test: Automated checks completed. Please verify if the top terms match the expected topics:", Colors.WARNING)
    print("- SpongeBob")
    print("- Computer Science")
    print("- Cooking")
    print("Note: The order of topics may not match the expected order.")

test_svd()
test_lsa()

Testing SVD implementation...

Test 1: Comparison with scipy's SVD
Max difference in U: 0.000000
Max difference in S: 0.000000
Max difference in V: 0.000000
Test 1 Passed!

Test 2: Reconstruction Error
Reconstruction error: 0.000000
Test 2 Passed!

Test 3: Small, known matrix

Max difference in U: 0.000000
Max difference in S: 0.000000
Max difference in V: 0.000000
Test 3 Passed: Small matrix SVD matches scipy results within tolerance

Test 4: Orthogonality of U and V
Max deviation from identity for U: 0.000000
Max deviation from identity for V: 0.000000
Test 4 Passed!

Test 5: Properties of singular values
Singular values:
[2.19869355 0.7975953  0.67087078 0.26096257]
Test 5 Passed!

Testing LSA implementation...

Our LSA document-topic matrix:
[[-0.65503505  0.08756422 -0.23535281]
 [-0.59749834  0.37524266 -0.13820511]
 [-0.42550279  0.30898413 -0.22702776]
 [-0.26376106  0.21932515  0.50295872]
 [-0.07958578  0.07452861  0.70868065]
 [-0.3018436  -0.08110509  0.45279548]
 [-0.32355

# Chapter 2: Neural Embeddings with Word2Vec - CBOW

## Pengantar Continuous Bag of Words (CBOW)

Continuous Bag of Words (CBOW) adalah teknik mendapatkan embedding berbasis neural network. CBOW adalah salah satu dari dua arsitektur model yang diperkenalkan dalam framework Word2Vec, bersama dengan Skip-gram.

Yang dilakukan CBOW adalah memprediksi kata target berdasarkan kata-kata konteks di sekitarnya. Sebagai contoh, pada kalimat dibawah inki

---

“𝙏𝙞𝙙𝙖𝙠 𝙖𝙙𝙖 𝙮𝙖𝙣𝙜 𝙥𝙚𝙙𝙪𝙡𝙞 𝙙𝙚𝙣𝙜𝙖𝙣 𝙣𝙖𝙨𝙞𝙗 𝙗𝙪𝙧𝙪𝙝 𝙨𝙚𝙡𝙖𝙢𝙖 𝙢𝙚𝙧𝙚𝙠𝙖 𝙗𝙞𝙨𝙖 𝙢𝙚𝙣𝙙𝙖𝙥𝙖𝙩𝙠𝙖𝙣 𝙠𝙚𝙥𝙪𝙖𝙨𝙖𝙣 𝙨𝙚𝙘𝙖𝙧𝙖 𝙞𝙣𝙨𝙩𝙖𝙣.”

-Squidward Q. Tentacles

---

arsitektur CBOW dapat mencoba memprediksi kata "buruh" berdasarkan kata konteks "dengan nasib" dan "selama mereka."

Metode ini memungkinkan kita untuk menangkap hubungan semantik antara kata-kata dengan merepresentasikannya sebagai vektor padat dalam ruang vektor. Dalam contoh kita, CBOW akan belajar mengasosiasikan “ buruh ” dengan konsep-konsep yang berkaitan.

Dalam tutorial ini, kita akan mengeksplorasi konsep-konsep dasar di balik CBOW, termasuk arsitektur dan proses pelatihannya. Kita akan mengimplementasikan CBOW dari awal menggunakan Python, mendapatkan pemahaman yang mendalam tentang bagaimana model ini mempelajari representasi kata dari kalimat-kalimat seperti contoh yang akan kita pelajari.

In [63]:
import torch
import random

torch.manual_seed(42)
random.seed(42)

## Persiapan dataset

Disini kita akan mempersiapkan data yang akan kita latih untuk mendapatkan embedding dengan model CBOW. Data yang digunakan adalah sintetis agar mudah untuk melakukan pengecekan. Kamu cukup jalankan saja dibawah ini.

In [64]:
def augment_dataset(sentences = [], num_augmented=200):
    templates = [
        "{character} {action} in {location}",
        "{character} and {character} {action} at the {location}",
        "{action} is important for {character} at the {location}",
        "{character} {action} with {character} near {location}",
        "oh barnacles {character} {action} again at the {location}",
        "who lives in a {location} under the sea {character}",
        "{character} loves to {action} more than anything else in {location}",
        "tartar sauce {character} forgot to {action} at the {location}",
        "{character} is ready to {action} at {location}",
        "im ready im ready says {character} before they {action} at {location}",
        "{character} and {character} have a {adjective} time when they {action}",
        "the best time to {action} is all the time says {character}",
        "{character} learns that {action} is not as easy as it looks",
        "money money money chants {character} while they {action}",
        "{character} tries to {action} but ends up making a {adjective} mess",
        "sweet victory {character} finally manages to {action} successfully",
        "{character} and {character} compete to see who can {action} better",
        "imagination {character} uses it to {action} in a {adjective} way",
        "{character} discovers a new way to {action} at {location}",
        "oh {exclamation} {character} cant believe they have to {action} again"
    ]

    characters = ['spongebob', 'patrick', 'squidward', 'mr krabs', 'sandy', 'plankton', 'gary', 'mrs puff', 'pearl', 'larry the lobster', 'mermaid man', 'barnacle boy', 'flying dutchman']
    actions = ['laughs', 'works', 'sings', 'dances', 'cooks', 'sleeps', 'plays', 'swims', 'jellyfishes', 'karates', 'bubbles', 'plans', 'dreams', 'practices', 'learns', 'teaches', 'explores', 'invents', 'relaxes', 'exercises']
    locations = ['krusty krab', 'bikini bottom', 'pineapple house', 'chum bucket', 'goo lagoon', 'jellyfish fields', 'boating school', 'barg n mart', 'weenie hut juniors', 'salty spitoon', 'treedome', 'rock bottom', 'kelp forest']
    adjectives = ['fun', 'crazy', 'silly', 'exciting', 'dangerous', 'hilarious', 'weird', 'amazing', 'unexpected', 'fantastic']
    exclamations = ['barnacles', 'fishpaste', 'tartar sauce', 'holy shrimp', 'great barrier reef']

    augmented_sentences = sentences.copy()

    for _ in range(num_augmented):
        template = random.choice(templates)
        new_sentence = template.format(
            character=random.choice(characters),
            action=random.choice(actions),
            location=random.choice(locations),
            adjective=random.choice(adjectives),
            exclamation=random.choice(exclamations)
        )
        augmented_sentences.append(new_sentence)

    return augmented_sentences

augmented_sentences = augment_dataset()

tokenized_sentences = [sentence.lower().split() for sentence in augmented_sentences]

vocab = list(set(word for sentence in tokenized_sentences for word in sentence))
vocab_size = len(vocab)

word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

print(f"Augmented vocabulary size: {vocab_size}")


Augmented vocabulary size: 147


## input-output pairs

Karena dalam CBOW kita akan memprediksi menggunakan konteks sekitar dari suatu kata, oleh karena itu disini kita akan mengimplementasikan bagaimana cara membuat data yang akan ditrain.

Feel free untuk mengikuti arahan dibawah atau mencoba cara sendiri.

1. Deklarasikan fungsi dan struktur data penyimpan
2. Lakukan iterasi terhadap documents dan words
3. Buat representasi konteks:
   - Inisialisasi vektor nol dengan panjang yang sama dengan ukuran vocab.
   - Vektor ini akan merepresentasikan kata-kata konteks sebagai bag of words.
4. Isi vektor konteks dengan vektor yang berada di window.
5. Ambil indeks kata target, ubah kata target menjadi indeksnya di dalam kosakata. Ini yang akan menjadi label yang kita coba prediksi dalam model CBOW.
6. Buat dan simpan pairs
7. Return pairs


In [65]:
def create_cbow_pairs(tokenized_documents, window_size=2):
    # Assumtion: Window Size kelipatan 2
    pairs = []
    for doc in tokenized_documents:
        for i, word in enumerate(doc):
          context_vector = torch.zeros(vocab_size)

          prev_context = [word_to_idx[word] for word in doc[(i - (window_size)//2) : i]]
          next_context = [word_to_idx[word] for word in doc[i + 1 : (i + (window_size)//2 + 1)]]
          contexts = prev_context + next_context
          for context in contexts:
            context_vector[context] += 1

          if (len(prev_context) + len(next_context)) == window_size:
              pairs.append((context_vector, word_to_idx[word]))
          else:
            continue

    return pairs

cbow_pairs = create_cbow_pairs(tokenized_sentences)
len(cbow_pairs)

1609

## Persiapan Model

Disini kita akan mempersiapkan matriks yang akan kita train dengan data yang sudah kita olah barusan, dimana:

W1 (Input ke hidden layer):
- W1 adalah matriks weight antara input layer dan lapisan hidden layer.
- Shapenya adalah ( vocabulary_size, embedding_size ).
- Setiap baris dalam W1 korespon dengan sebuah kata dalam vocab.
- Setelah training, W1 berisi embedding kata yang telah dipelajari.
- Embeddings ini yang harapannya akan menangkap makna semantik dari kata-kata.

W2 (Hidden layer ke output):
- W2 adalah matriks weight antara hidden layer dan output layer.
- Shapenya adalah (embedding_size, vocabulary_size).
- Matriks ini membantu dalam memprediksi distribusi probabilitas atas semua kata dalam vocab.

Masing-masing silakan inisialisasi dengan matriks random.

In [66]:
embedding_dim = 50

W1 = torch.randn(vocab_size, embedding_dim, requires_grad=True)
W2 = torch.randn(embedding_dim, vocab_size, requires_grad=True)

## Training

Setelah menyiapkan data, kita perlu melatih model CBOW. Hal ini berarti kita harus implementasi penerusan jaringan saraf dan training loop secara manual.

### Forward Pass:
   - Fungsi ini merepresentasikan arsitektur NN untuk CBOW.
   - Langkah-langkah dalam proses forward:
     
     - Kalikan matriks konteks dengan W1 untuk mendapatkan representasi hidden layer.
     
     - Kalikan kembali hidden layer dengan W2 untuk mendapatkan output score untuk setiap kata dalam vocab.
   - Kembalikan output.

In [67]:
def forward(context, W1, W2):
  # TODO

  hidden_layer = context @ W1

  output = hidden_layer @ W2

  return output

### Training Loop
Setelah mengimplementasikan function forward, kita dapat memulai membuat training loopnya. Pertama kita juga harus siapkan loss function, untuk program ini kita dapat gunakan CrossEntropyLoss. Selanjutnya kita buat loop untuk setiap epoch, yang dimana setiap epoch kita ingin update weights dari semua data training, sehingg setiap loop untuk pasangan konteks dan target dapat kita lakukan:

1. Panggil fungsi `forward` dengan matriks konteks dan bobot saat ini.
2. Hitung loss
3. Lakukan backward pass
4. Update weight
   - Gunakan gradien dan learning rate untuk memperbarui W1 dan W2.
5. Reset kembali gradien
6. Epoch selanjutnya

kemudian return kembali W1 dan W2

In [68]:
def train_cbow(cbow_pairs, W1, W2, num_epochs=100, learning_rate=0.01):
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        total_loss = 0
        for context, target_index in cbow_pairs:
            context_tensor = torch.tensor(context, dtype=torch.float32)
            target_tensor = torch.tensor(target_index, dtype=torch.long)

            # TODO: Forward pass
            output = forward(context_tensor, W1, W2)
            # TODO: Compute loss
            loss = loss_fn(output, target_tensor)

            # TODO: Backward pass
            loss.backward()

            # TODO: Update weights

            with torch.no_grad():
              W1 -= learning_rate * W1.grad
              W1.grad.zero_()

              W2 -= learning_rate * W2.grad
              W2.grad.zero_()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(cbow_pairs)}")

    return W1, W2


W1, W2 = train_cbow(cbow_pairs, W1, W2)

<ipython-input-68-531617fa494d>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  context_tensor = torch.tensor(context, dtype=torch.float32)


Epoch 1, Loss: 17.244158229350955
Epoch 2, Loss: 7.1060341247561505
Epoch 3, Loss: 3.9551063357717084
Epoch 4, Loss: 2.6184376582920295
Epoch 5, Loss: 1.8932258064123466
Epoch 6, Loss: 1.4756565577602552
Epoch 7, Loss: 1.2076955248965695
Epoch 8, Loss: 1.02352085748933
Epoch 9, Loss: 0.8910857202432694
Epoch 10, Loss: 0.7939291941787796
Epoch 11, Loss: 0.723417457510284
Epoch 12, Loss: 0.6720385171341876
Epoch 13, Loss: 0.6329504062033922
Epoch 14, Loss: 0.6023338082290444
Epoch 15, Loss: 0.5778357480100932
Epoch 16, Loss: 0.5578441980205562
Epoch 17, Loss: 0.5414784614244802
Epoch 18, Loss: 0.5281954567173242
Epoch 19, Loss: 0.5175235916300022
Epoch 20, Loss: 0.5089790097949802
Epoch 21, Loss: 0.5021180593275874
Epoch 22, Loss: 0.49660416416217423
Epoch 23, Loss: 0.4922533168428554
Epoch 24, Loss: 0.48885032179879934
Epoch 25, Loss: 0.4860692167227033
Epoch 26, Loss: 0.4837020929057242
Epoch 27, Loss: 0.4816397378575361
Epoch 28, Loss: 0.47981691211696303
Epoch 29, Loss: 0.47819036488

### Penggunaan

Setelah melatih model CBOW, kita dapat mendapatkan word embedding yang telah dipelajari untuk menemukan kata-kata yang mirip secara semantik.

Kita disini bisa detach matriks W1 dari grafik komputasi untuk menggunakannya sebagai embedding.

Matriks ini berisi vektor kata yang telah dipelajari untuk setiap kata dalam vocab kita.

In [69]:
word_embeddings = W1.detach()
word_embeddings.shape

torch.Size([147, 50])

Dibawah ini adalah contoh untuk mencari kata yang serupa.

In [70]:
def get_similar_words(word, word_to_idx, idx_to_word, word_embeddings, top_k=5):
    if word not in word_to_idx:
        return []

    word_idx = word_to_idx[word]
    word_embedding = word_embeddings[word_idx]

    similarities = torch.matmul(word_embeddings, word_embedding)
    top_indices = torch.argsort(similarities, descending=True)[1:top_k+1]

    return [idx_to_word[idx.item()] for idx in top_indices]



word = "krusty"
similar_words = get_similar_words(word, word_to_idx, idx_to_word, word_embeddings)
print(f"Words similar to '{word}': {similar_words}")

Words similar to 'krusty': ['swims', 'while', 'fishpaste', 'bikini', 'sings']
